In [46]:
import cv2
import numpy as np

In [47]:
F0 = cv2.imread("../Images/sim_Landsat_t1.tif")
C0 = cv2.imread("../Images/sim_MODIS_t1.tif")
C1 = cv2.imread("../Images/sim_MODIS_t2.tif")

In [48]:
type(F0)

numpy.ndarray

In [49]:
F0.shape

(150, 150, 3)

In [50]:
windowSize = 3
padAmount = windowSize//2
print(padAmount)

1


In [51]:
padded_F0 = np.pad(F0, [(padAmount, padAmount), (padAmount, padAmount), (0, 0)], mode='constant', constant_values=0)

In [52]:
padded_F0.shape

(152, 152, 3)

In [53]:
F0_test = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])

In [54]:
def padArray(arr, windowSize):
    padAmount = windowSize//2
    return np.pad(arr, [(padAmount, padAmount), (padAmount, padAmount), (0, 0)], mode='constant', constant_values=0)

In [55]:
F0_test.shape

(2, 2, 3)

In [56]:
F0_test.shape

(2, 2, 3)

In [57]:
np.set_printoptions(linewidth=np.inf)

In [58]:
padded_F0_test = padArray(F0_test, 3)

In [59]:
padded_F0_test.shape

(4, 4, 3)

In [60]:
print(padded_F0_test)

[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 1  2  3]
  [ 4  5  6]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 7  8  9]
  [10 11 12]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]


In [61]:
print(padAmount)

1


In [62]:
i, j = 1, 1
window = padded_F0_test[i-windowSize//2:i+windowSize//2+1, j-windowSize//2:j+windowSize//2+1]
print(window)

[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 1  2  3]
  [ 4  5  6]]

 [[ 0  0  0]
  [ 7  8  9]
  [10 11 12]]]


In [63]:
for i in range(padAmount, len(padded_F0_test)-padAmount):
    for j in range(padAmount, len(padded_F0_test[0])-padAmount):
        window = padded_F0_test[i-windowSize//2:i+windowSize//2+1, j-windowSize//2:j+windowSize//2+1]
        print(window)
        print("NEXT WINDOW")
    

[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 1  2  3]
  [ 4  5  6]]

 [[ 0  0  0]
  [ 7  8  9]
  [10 11 12]]]
NEXT WINDOW
[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 1  2  3]
  [ 4  5  6]
  [ 0  0  0]]

 [[ 7  8  9]
  [10 11 12]
  [ 0  0  0]]]
NEXT WINDOW
[[[ 0  0  0]
  [ 1  2  3]
  [ 4  5  6]]

 [[ 0  0  0]
  [ 7  8  9]
  [10 11 12]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]
NEXT WINDOW
[[[ 1  2  3]
  [ 4  5  6]
  [ 0  0  0]]

 [[ 7  8  9]
  [10 11 12]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]
NEXT WINDOW


# TESTING PREDICTION

In [64]:
import cv2
import numpy as np

In [65]:
# Set the size of the moving window in which the search for similar pixels 
# is performed
windowSize = 3

# Set the path where the results should be stored
path = 'STARFM_demo/'

# Set to True if you want to decrease the sensitivity to the spectral distance
logWeight = False

# If more than one training pairs are used, set to True
temp = False

# The spatial impact factor is a constant defining the relative importance of 
# spatial distance (in meters)
# Take a smaller value of the spatial impact factor for heterogeneous regions 
# (e.g. A = 150 m)
spatImp = 150 

# increasing the number of classes limits the number of similar pixels
numberClass = 4

# Set the uncertainty value for the fine resolution sensor
# https://earth.esa.int/web/sentinel/technical-guides/sentinel-2-msi/performance 
uncertaintyFineRes = 0.03

# Set the uncertainty value for the coarse resolution sensor
# https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-3-olci/validation
uncertaintyCoarseRes = 0.03

# Other global variables
mid_idx = (windowSize**2)//2
specUncertainty = np.sqrt(uncertaintyFineRes**2 + uncertaintyCoarseRes**2)
tempUncertainty = np.sqrt(2*uncertaintyCoarseRes**2)

# Set the size of the slices in which to divide the image
# This number should be multiple of the image height and not bigger than it
# Use bigger size for small images
sizeSlices = 150

# The depth of zeros to pad around each numpy array
padAmount = windowSize//2

### Padding array

In [66]:
def padArray(arr, windowSize):
    return np.pad(arr, [(padAmount, padAmount), (padAmount, padAmount), (0, 0)], mode='constant', constant_values=0)

In [67]:
F0_test = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
padded_F0_test = np.pad(F0_test, pad_width=windowSize//2, mode='constant', constant_values=0)
padded_F0_test

array([[0., 0., 0., 0., 0.],
       [0., 1., 2., 3., 0.],
       [0., 4., 5., 6., 0.],
       [0., 7., 8., 9., 0.],
       [0., 0., 0., 0., 0.]])

In [68]:
C0_test = np.array([[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]])
padded_C0_test = np.pad(C0_test, pad_width=windowSize//2, mode='constant', constant_values=0)
padded_C0_test

array([[0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0.]])

In [69]:
C1_test = np.array([[1.0, 2.0, 3.0], [1.0, 2.0, 3.0], [1.0, 2.0, 3.0]])
padded_C1_test = np.pad(C1_test, pad_width=windowSize//2, mode='constant', constant_values=0)
padded_C1_test

array([[0., 0., 0., 0., 0.],
       [0., 1., 2., 3., 0.],
       [0., 1., 2., 3., 0.],
       [0., 1., 2., 3., 0.],
       [0., 0., 0., 0., 0.]])

In [70]:
F1_test = C1_test.copy()
F1_test

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

### Spectral Distance

In [71]:
def spectral_distance(F0_window, C0_window):
    spectral_difference_window = F0_window - C0_window
    spectral_distance_window = 1/(abs(spectral_difference_window) + 1.0)
    return spectral_difference_window, spectral_distance_window

### Temporal Distance

In [72]:
def temporal_distance(C0_window, C1_window):
    temp_difference_window = C1_window - C0_window
    temp_distance_window = 1/(abs(temp_difference_window) + 1.0)
    return temp_difference_window, temp_distance_window

### Spatial Distance

In [73]:
def spatial_distance(F0_window):
    coord = np.sqrt((np.mgrid[0:windowSize,0:windowSize]-windowSize//2)**2)
    spat_dist = np.sqrt(((0-coord[0])**2+(0-coord[1])**2))
    rel_spat_dist = spat_dist/spatImp + 1.0 # relative spatial distance
    rev_spat_dist = 1/rel_spat_dist # relative spatial distance reversed
    flat_spat_dist = np.ravel(rev_spat_dist)
    # spatial_distance_window = np.repeat(flat_spat_dist[:, np.newaxis], 3, axis=1)
    return flat_spat_dist

### Combination Distance

In [74]:
def combination_distance(spectral_distance_window, temporal_distance_window, spatial_distance_window):
    if logWeight == True:
        spectral_distance_window = np.log(spectral_distance_window+1)
        temporal_distance_window = np.log(temporal_distance_window + 1)
    combination_distance_window = spectral_distance_window*temporal_distance_window*spatial_distance_window
    return combination_distance_window

### Similarity Threshold

In [75]:
def similarity_threshold(F0_window):
    F0_window = np.where(F0_window==0, np.nan, F0_window)
    # print(F0_window)
    # print("IN FUNCTION: similarity_threshold")
    # print("similar_pixels :", F0_window)
    st_dev = np.nanstd(F0_window, axis=0)
    # print("st_dev :", st_dev)
    # print(st_dev)
    sim_threshold = st_dev*2/numberClass
    # print("sim_threshold :", sim_threshold)
    # print(sim_threshold)
    return sim_threshold

### Similarity Pixels

In [76]:
def similarity_pixels(F0_window):
    sim_threshold = similarity_threshold(F0_window)
    # print("IN FUNCTION: similarity_pixels")
    # print("sim_threshold :", sim_threshold)
    similar_pixels = np.where(abs(F0_window - F0_window[mid_idx]) <= sim_threshold, 1, 0) 
    # print("similar_pixels :", similar_pixels)
    return similar_pixels

### Filtering

In [77]:
def filtering(F0_window, spectral_distance_window, temporal_distance_window, spectral_difference_window, temporal_difference_window):
    similar_pixels = similarity_pixels(F0_window)
    # print("IN FUNCTION: filtering")
    # print("similar_pixels :", similar_pixels)
    max_spec_dist = abs(spectral_difference_window)[mid_idx] + specUncertainty + 1
    max_temp_dist = abs(temporal_difference_window)[mid_idx] + tempUncertainty + 1  
    # print("max_spec_dist", max_spec_dist)
    # print("max_temp_dist", max_temp_dist)
    spec_filter = np.where(spectral_distance_window>1.0/max_spec_dist, 1, 0)
    # print("spec_filter :", spec_filter)
    st_filter = spec_filter

    # print(st_filter)

    if temp == True:
        temp_filter = np.where(temporal_distance_window>1.0/max_temp_dist, 1, 0)
        st_filter = spec_filter*temp_filter

    similar_pixels = similar_pixels*st_filter
    return similar_pixels

### Weights

In [78]:
def weighting(spec_dist, temp_dist, comb_dist, similar_pixels_filtered):
    # Assign max weight (1) when the temporal or spectral distance is zero
    # print("WEIGHTS FUNCTION")
    zero_spec_dist = np.where(spec_dist[mid_idx] == 1, 1, 0)
    # print("zero_spec_dist: ", zero_spec_dist)
    
    zero_temp_dist = np.where(temp_dist[mid_idx] == 1, 1, 0)
    # print("zero_temp_dist: ", zero_temp_dist)

    zero_dist_mid = np.where((zero_spec_dist == 1), zero_spec_dist, zero_temp_dist)
    # print("zero_dist_mid: ", zero_dist_mid)

    shape = np.subtract(spec_dist.shape,(0,1))
    # print("shape: ", shape)

    zero_dist = np.zeros(shape[1])
    # print("zero_dist", zero_dist)
    zero_dist = np.insert(zero_dist, [mid_idx], zero_dist_mid, axis=0)
    # print("zero_dist", zero_dist)
    weights = np.where((np.sum(zero_dist) == 1), zero_dist, comb_dist)
    # print("weights", weights)
    # Calculate weights only for the filtered spectrally similar pixels
    weights_filt = weights*similar_pixels_filtered
    # print("weights_filt", weights_filt)
    
    # Normalize weights
    norm_weights = weights_filt/(np.sum(weights_filt))
                            
    # print ("Done weighting!", norm_weights)
    
    return norm_weights

## Prediction

In [79]:
for i in range(padAmount, len(padded_F0_test)-padAmount):
    for j in range(padAmount, len(padded_F0_test[0])-padAmount):
        F0_window = padded_F0_test[i-windowSize//2:i+windowSize//2 + 1, j-windowSize//2:j+windowSize//2 + 1].flatten()
        C0_window = padded_C0_test[i-windowSize//2:i+windowSize//2 + 1, j-windowSize//2:j+windowSize//2 + 1].flatten()
        C1_window = padded_C1_test[i-windowSize//2:i+windowSize//2 + 1, j-windowSize//2:j+windowSize//2 + 1].flatten()

        # print(f"Window around point ({i}, {j})")
        # print("F0_window")
        # print(F0_window)

        # print("C0_window")
        # print(C0_window)

        # print("C1_window")
        # print(C1_window)

        # Spectral Difference/Distance
        spectral_difference_window, spectral_distance_window = spectral_distance(F0_window, C0_window)

        # print("spectral_difference_window")
        # print(spectral_difference_window)
        # print("spectral_distance_window")
        # print(spectral_distance_window)
        
        # Temporal Difference/Distance
        temporal_difference_window, temporal_distance_window = temporal_distance(C0_window, C1_window)
        # print("temporal_difference_window")
        # print(temporal_difference_window)
        # print("temporal_distance_window")
        # print(temporal_distance_window)
    
        # Spatial Distance
        spatial_distance_window = spatial_distance(F0_window)
        # print("spatial_distance_window")
        # print(spatial_distance_window)

        combination_distance_window = combination_distance(spectral_distance_window, temporal_distance_window, spatial_distance_window)
        # print("combination_distance_window")
        # print(combination_distance_window)

        # print(F0_window)

        similar_pixels_window = filtering(F0_window, spectral_distance_window, temporal_distance_window, spectral_difference_window, temporal_difference_window)
        # print("similar_pixels_window")
        # print(similar_pixels_window)

        weights = weighting(spectral_distance_window, temporal_distance_window, combination_distance_window, similar_pixels_window)

        # print("weights")
        # print(weights)


        pred_refl = F0_window + temporal_difference_window
        weighted_pred_refl = np.sum(pred_refl*weights)   
        
        F1_test[i-padAmount][j-padAmount] = weighted_pred_refl
F1_test


        
        
    

        


1.0
3.0
5.0
4.0
4.574144486692015
6.7173396674584325
7.0
9.0
11.0


array([[ 1.        ,  3.        ,  5.        ],
       [ 4.        ,  4.57414449,  6.71733967],
       [ 7.        ,  9.        , 11.        ]])

## Paper Link
https://opus.bibliothek.uni-augsburg.de/opus4/frontdoor/deliver/index/docId/78805/file/STARFM_paper.pdf